# John Doll
## Question 1

In [11]:
# function to scrape given website and return data about football teams
def scrape_football_data (url):
    import requests   
    import re
    import pandas as pd
    r = requests.get(url)
    html_code = r.text
    
    # get team names
    team_hits = re.finditer("(teamnum=[0-9]{1,2} TARGET=\"_BLANK\">)(.+?)(</A>)", html_code)
    team_matches = [m.group(2) for m in team_hits]
    home_teams = []
    away_teams = []
    winning_teams = []

    # add team names to winning team and either home or away team
    for m in range(0, len(team_matches) - 2):
        if (team_matches[m][:3] == "<B>"):
            team_matches[m] = team_matches[m][3:-4]
            winning_teams.append(team_matches[m])
        if (m % 2 == 0):
            home_teams.append(team_matches[m])
        else:
            away_teams.append(team_matches[m])

    # get prediction rates
    per_hits = re.finditer("(<NOBR>)(<B>){0,1}([0-9]{1,2})(</B>){0,1}(%){1}", html_code)
    percent = [p.group(3) for p in per_hits]
    home_pred = []
    away_pred = []

    # split prediction rates into either home or away prediction
    for p in range(0, len(percent)):
        if (p % 2 == 0):
            home_pred.append(percent[p])
        else:
            away_pred.append(percent[p])

    # add all data to data frame and return it
    df = pd.DataFrame({"Home_Team":home_teams, "Away_team":away_teams, "Winner_Team":winning_teams, "Prediction_Home_Team":home_pred, "Prediction_Away_Team":away_pred})
    return df

# get football data frame from function
football_df = scrape_football_data("https://probabilityfootball.com/picks.html?1487349677&username=AVERAGES&weeknum=5")

## Question 2

In [12]:
import pandas as pd
weeks = []
# loop over every week in the function and add that to an array of data frames
for week in range(5, 22):
    football_df = scrape_football_data(f"https://probabilityfootball.com/picks.html?1487349677&username=AVERAGES&weeknum={week}")
    weeks.append(football_df)

# add all the weeks to one data frame
season = pd.concat(weeks)

## Question 3

In [13]:
import scipy.stats as ss
import numpy as np
# create column for winner predictions
predictions_winner = []

# assign all winner predictions
for pred in range(0, 256):
    if (season.iloc[pred, 1] == season.iloc[pred, 2]):
        predictions_winner.append(season.iloc[pred, 4])
    else:
        predictions_winner.append(season.iloc[pred, 3])

# convert predictions array to numeric values for ttest
predictions_winner = np.array(predictions_winner).astype(float)

# add winner predictions to data frame
season = season.assign(predictions_winner = predictions_winner)

# perform t-test 
ss.ttest_1samp(predictions_winner, 50)


Ttest_1sampResult(statistic=7.354975028437655, pvalue=2.6067816471438056e-12)

The resulting p-value is 2.61 x $10^{-12}$ which is significant with alpha level 0.05. Based on this result, we trust the experts to correctly pick the correct winner more than a coin flip on average. I feel that an unbiased coin is a fair comparison as it is just the same as someone randomly picking who will the game, and since we are testing if the experts are better than random at picking, this comparison makes sense.